In [ ]:
import torch

In [1]:
with open('wizard_of_oz.txt','r',encoding='utf-8') as f:
    text = f.read()
    
print(text[:200])

The Project Gutenberg eBook of Dorothy and the Wizard in Oz
    
This ebook is for the use of anyone anywhere in the United States and
most other parts of the world at no cost and with almost no rest


In [2]:
import torch
print("PyTorch version:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
print("CUDA version:", torch.version.cuda)
print("Device name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU found")


PyTorch version: 2.6.0+cu126
CUDA available: True
CUDA version: 12.6
Device name: NVIDIA GeForce RTX 3050 Laptop GPU


In [3]:
chars = sorted(set(text))

In [4]:
len(chars) , print(chars)

['\n', ' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '—', '‘', '’', '“', '”', '•', '™', '\ufeff']


(92, None)

In [5]:
# now we will use tokenizer here for chars ......
#tokenizer consist of a encoder and an decoder , it actually gonna convert eaach element of chars into an integer.....


In [29]:
#tokenizer
string_to_int = {ch:i for i , ch in enumerate(chars)}
# int_to_string = {i:ch for i, ch in enumerate(chars)}

encode = lambda s : [string_to_int[c] for c in s ]
# decode = lambda l : ''.join([int_to_string[i] for i in l ])
int_to_string = {i: chr(32 + (i % 94)) for i in range(500)}
decode = lambda l: ''.join([int_to_string.get(i, '?') for i in l])


print(encode('hello'))
print(decode(encode('hello')))
print(decode([65, 62, 69, 69, 72]))

[65, 62, 69, 69, 72]
a^eeh
a^eeh


In [7]:
print(encode(text)[:100])



[91, 48, 65, 62, 1, 44, 75, 72, 67, 62, 60, 77, 1, 35, 78, 77, 62, 71, 59, 62, 75, 64, 1, 62, 30, 72, 72, 68, 1, 72, 63, 1, 32, 72, 75, 72, 77, 65, 82, 1, 58, 71, 61, 1, 77, 65, 62, 1, 51, 66, 83, 58, 75, 61, 1, 66, 71, 1, 43, 83, 0, 1, 1, 1, 1, 0, 48, 65, 66, 76, 1, 62, 59, 72, 72, 68, 1, 66, 76, 1, 63, 72, 75, 1, 77, 65, 62, 1, 78, 76, 62, 1, 72, 63, 1, 58, 71, 82, 72, 71]


In [8]:
import torch

data = torch.tensor(encode(text), dtype=torch.long)
print(data[:100])

tensor([91, 48, 65, 62,  1, 44, 75, 72, 67, 62, 60, 77,  1, 35, 78, 77, 62, 71,
        59, 62, 75, 64,  1, 62, 30, 72, 72, 68,  1, 72, 63,  1, 32, 72, 75, 72,
        77, 65, 82,  1, 58, 71, 61,  1, 77, 65, 62,  1, 51, 66, 83, 58, 75, 61,
         1, 66, 71,  1, 43, 83,  0,  1,  1,  1,  1,  0, 48, 65, 66, 76,  1, 62,
        59, 72, 72, 68,  1, 66, 76,  1, 63, 72, 75,  1, 77, 65, 62,  1, 78, 76,
        62,  1, 72, 63,  1, 58, 71, 82, 72, 71])


In [9]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


In [10]:
# block 1-> [......] has 8 elements
# block 2-> [......] has 8 elements
# .
# .
# .
# block n-> [......] has 8 elements

# there are n number of blocks with size 8 , i.e the batch_size = n , block_size=8

In [11]:
block_size = 8
batch_size  = 4

n=int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

x=train_data[:block_size]
y=train_data[:block_size+1]

for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print("when input is " ,context,'target is ',target)

when input is  tensor([91]) target is  tensor(91)
when input is  tensor([91, 48]) target is  tensor(48)
when input is  tensor([91, 48, 65]) target is  tensor(65)
when input is  tensor([91, 48, 65, 62]) target is  tensor(62)
when input is  tensor([91, 48, 65, 62,  1]) target is  tensor(1)
when input is  tensor([91, 48, 65, 62,  1, 44]) target is  tensor(44)
when input is  tensor([91, 48, 65, 62,  1, 44, 75]) target is  tensor(75)
when input is  tensor([91, 48, 65, 62,  1, 44, 75, 72]) target is  tensor(72)


In [12]:
# Sample data (example for demonstration)
# data = torch.arange(100)

# Train-Validation Split
n = int(0.8 * len(data))
train_data = data[:n]
val_data = data[n:]


def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(0, len(data) - block_size, (batch_size,))
    #ix = index
    
    print(f"indices: {ix}")
    
    x = torch.stack([data[i : i + block_size] for i in ix])
    y = torch.stack([data[i + 1 : i + block_size + 1] for i in ix])  # Shifted by 1 for next token prediction
    
    return x, y

# Fetch data
x, y = get_batch('train')
print(f"Inputs  : {x}")
print(f"Targets : {y}")


indices: tensor([154706, 173511, 161096, 118434])
Inputs  : tensor([[75,  1, 61, 75, 58, 64, 72, 71],
        [70, 76,  1, 77, 65, 58, 77,  1],
        [ 1, 77, 65, 58, 77,  1, 65, 58],
        [58, 71, 72, 77, 65, 62, 75,  1]])
Targets : tensor([[ 1, 61, 75, 58, 64, 72, 71,  1],
        [76,  1, 77, 65, 58, 77,  1, 80],
        [77, 65, 58, 77,  1, 65, 58, 61],
        [71, 72, 77, 65, 62, 75,  1, 72]])


In [13]:
import torch.nn as nn
import torch.nn.functional as F

# logits are basically bunch of floating point numbers which are unnormalized
#Logits are the raw, unnormalized output values produced by the last layer of a neural network before passing through an activation function like softmax or sigmoid.
#Why Use Logits?

Logits simplify the computation of loss functions like:

nn.CrossEntropyLoss() (automatically applies softmax inside).
 nn.BCEWithLogitsLoss() (automatically applies sigmoid inside).
 Using logits directly in these loss functions is numerically more stable than applying softmax or sigmoid separately.

 In short, logits are just the network’s "raw predictions" before being converted into meaningful probabilities. 🚀

In [14]:
import torch
import torch.nn as nn

# Sample logits output
logits = torch.tensor([2.0, 1.5, -0.5])

# Convert logits to probabilities using softmax
probabilities = nn.Softmax(dim=0)(logits) # logits are basically distribution of what we wanna predict

print("Logits:", logits)
print("Probabilities:", probabilities)


Logits: tensor([ 2.0000,  1.5000, -0.5000])
Probabilities: tensor([0.5922, 0.3592, 0.0486])


Token Embedding in NLP
A token embedding is a numerical representation of a text token (like a word, subword, or character) in a continuous vector space. It’s a key concept in Natural Language Processing (NLP) that allows models like Transformers, BERT, and GPT to understand text data efficiently.

How Does Token Embedding Work?
Tokenization:

First, text is split into smaller units called tokens.
These tokens can be words, subwords, or characters depending on the tokenizer used.
Example:
css
Copy
Edit
"I love PyTorch" → ["I", "love", "Py", "##Torch"]
Embedding Layer:
Each token is mapped to a high-dimensional vector through an embedding matrix.

Suppose the vocabulary size is 10,000 and each token gets embedded into a 768-dimensional vector.
The embedding matrix will have a shape of (10000, 768).
Vector Representation:
Each token is assigned a unique vector from this matrix, trained to capture the token's meaning, context, and relationships.



In [15]:
import torch
import torch.nn as nn

# Example: Vocabulary size = 5000, Embedding dimension = 300
embedding_layer = nn.Embedding(num_embeddings=5000, embedding_dim=300)

# Sample input tokens (batch size = 2, sequence length = 4)
input_tokens = torch.tensor([[1, 45, 203, 4], 
                             [12, 344, 23, 654]])

# Forward pass to get embeddings
embeddings = embedding_layer(input_tokens)

print("Embedding shape:", embeddings.shape)  # Output: torch.Size([2, 4, 300])


Embedding shape: torch.Size([2, 4, 300])


1) def __init__(self, vocab_size):
    super().__init__()
    self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

The nn.Embedding() layer maps each token (word index) to a dense vector representation.
Here, both the input dimension and the embedding dimension are set to vocab_size.
This unusual choice means that each token is effectively mapped to a one-hot encoded vector during training.
✅ Why vocab_size as the embedding size?

Each token's embedding directly represents logits (raw scores) for predicting the next token.
Instead of adding an extra linear layer, this design simplifies prediction for the next token.


2) def forward(self, index, targets):
    logits = self.token_embedding_table(index)  # Shape: [B, T, C]

index → Tensor of token indices (shape [B, T], where B = batch size, T = sequence length).
The nn.Embedding() layer outputs a 3D tensor of shape [B, T, C] where:
B = Batch size
T = Sequence length
C = Number of classes (equals vocab_size)
➡️ Each position's embedding directly represents the logits for predicting the next token.



3)B, T, C = logits.shape
logits = logits.view(B * T, C)
targets = targets.view(B * T)


The logits are reshaped from [B, T, C] to [B * T, C] to flatten the batch and time dimensions for easier loss calculation.
The targets are similarly reshaped to [B * T].
✅ Why Reshape?

F.cross_entropy() expects inputs in the shape [N, C] where N is the total number of predictions and C is the number of classes.


4)loss = F.cross_entropy(logits, targets)
F.cross_entropy() is used for classification tasks. It combines both:
LogSoftmax (to convert logits into probabilities).
Negative Log Likelihood Loss (to compute the actual loss).
This step trains the model to predict the next token given the current one.






In [32]:


block_size=8
batch_size=4
max_iters=10000
learning_rate=3e-4

In [17]:


class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        # Embedding layer maps each token to a vector of size 'vocab_size'
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
        
    def forward(self, index, targets=None):
        # Look up embeddings for the input tokens
        logits = self.token_embedding_table(index)  # Shape: [B, T, C]
        
        if targets is None:
           loss = None
        else:
            # Reshape logits for cross-entropy loss
            B, T, C = logits.shape
            logits = logits.view(B * T, C)  # Flatten for batch processing
        
            # Reshape targets to match logits shape
            targets = targets.view(B * T)
        
            #we can also use reshape but they both have their own advantages
            #view() reshapes the tensor to match PyTorch's F.cross_entropy() input requirements.
        
            # Compute the loss using CrossEntropyLoss
            loss = F.cross_entropy(logits, targets)
        
        return logits ,loss
    
    def generate(self,index,max_new_tokens):
        
        # index → Initial sequence of token indices (starting prompt).
        # max_new_tokens → Number of new tokens to generate.
        
        for _ in range(max_new_tokens):
        # The loop runs max_new_tokens times, generating one new token at each step.
            
            # Get the logits for the last token in the sequence
            logits ,loss = self.forward(index)  # Shape: [B, T, C]
            logits = logits[:, -1, :]  # Select the last token only (shape [B, C])
            
            # Apply softmax to convert logits to probabilities
            probs = F.softmax(logits, dim=-1)  # Shape: [B, C]
            
            # Sample the next token from the probability distribution
            next_token = torch.multinomial(probs, num_samples=1)  # Shape: [B, 1]
            
            
            # Append the predicted token to the sequence
            index = torch.cat([index, next_token], dim=1)  # Shape: [B, T+1]

        return index
    





In [31]:
model= BigramLanguageModel(vocab_size=500)
m = model.to(device)

context = torch.zeros((1,1) , dtype=torch.long , device=device)
generated_chars = decode(m.generate(context,max_new_tokens=500)[0].tolist())
print(generated_chars)

 gK-E/"WOICqsn83c5O#/\_'D3xPt\^Q!])s,{XDxY{|,hVE.^<1v1+KMMY:^1IHMs6+e04o [;G^#cQ=AY%FAyoG}<sz]a!ciG9nqv(/w/ m#UH+q31,{C=RmzQ&g$%eylcv9.C1q!E@{`)o.t*^.0:\ftkg=4gLw@6.^%4E?0"RsSu=oT0c7aHW@eY:<[Z{}<%'v}'om.G`8a"Fq4nl7td!7ceOb11-CP{)CmbM*uz4,k[qn B!ao"&*TlUJu4grXl0'(?|7 Z1Kb@y|/o;!7p|L-t4s8fSm<'htXv_**pP2$c=0\EUvN(Ese,dheg<M@$f] /(DZy5HJ5Gl8>58o0Rb?=LfQ{;9Q=Yh|,qby64+Jep&g)>H3(+bDlfHf;858lfJJ,65CCZT'_LNwGpzRf({bWe-_#v'2RWV6t=vFkTx41,5OqKogBj^5d+;Mt@`5A<NLP1Yw'_'4GfB;O{c85|N]T/v9w?.666u4q$-qu\FA"Wlm1<


In [19]:
print("Max index in int_to_string:", max(int_to_string.keys()))
print("Generated token sequence:", m.generate(context, max_new_tokens=500)[0].tolist())


Max index in int_to_string: 91
Generated token sequence: [0, 462, 156, 110, 33, 325, 329, 88, 205, 490, 224, 150, 52, 74, 131, 355, 422, 331, 294, 211, 255, 400, 364, 113, 246, 357, 89, 221, 57, 7, 351, 321, 226, 360, 468, 293, 434, 308, 148, 362, 252, 240, 207, 487, 138, 85, 443, 115, 36, 49, 183, 12, 88, 464, 39, 307, 392, 150, 54, 274, 393, 335, 235, 294, 383, 8, 399, 230, 477, 315, 126, 27, 260, 113, 242, 191, 124, 317, 139, 386, 191, 294, 63, 434, 303, 372, 247, 362, 41, 41, 238, 81, 83, 38, 466, 165, 210, 209, 172, 399, 308, 408, 127, 206, 274, 376, 257, 474, 479, 291, 92, 93, 373, 174, 77, 477, 348, 408, 98, 30, 205, 207, 44, 110, 129, 21, 272, 78, 215, 192, 299, 41, 198, 436, 125, 498, 444, 3, 249, 491, 195, 445, 81, 263, 441, 435, 194, 85, 87, 452, 472, 115, 214, 75, 32, 123, 309, 74, 95, 65, 14, 100, 18, 162, 477, 12, 415, 493, 75, 388, 228, 193, 173, 36, 338, 480, 435, 139, 370, 189, 413, 371, 283, 490, 178, 476, 54, 208, 81, 217, 209, 432, 185, 70, 14, 175, 89, 69, 235, 281

1) logits = self.token_embedding_table(index)  # Shape: [B, T, C]
logits = logits[:, -1, :]  # Shape: [B, C]

The model generates logits for all tokens in the sequence ([B, T, C]).
Since we're only interested in the next token prediction, .select() or slicing with [:, -1, :] extracts the logits for the last token in the sequence.
The shape becomes [B, C], where:
B = Batch size
C = Vocabulary size (each class corresponds to a token in the vocabulary).

2) probs = torch.softmax(logits, dim=-1)  # Shape: [B, C]

Softmax converts the logits into a probability distribution over the vocabulary.
Each value now represents the likelihood of a particular token being the next one.

3) next_token = torch.multinomial(probs, num_samples=1)  # Shape: [B, 1]
torch.multinomial() samples one token based on the generated probabilities.
This introduces randomness, making the text more natural and varied.
If you want deterministic predictions, you could use .argmax() instead:

next_token = torch.argmax(probs, dim=-1, keepdim=True)

4)index = torch.cat([index, next_token], dim=1)  # Shape: [B, T+1]
The newly generated token is added to the original sequence using torch.cat().
This updated index is fed back into the loop for the next prediction.

5)return index

After generating max_new_tokens, the final sequence (including the newly added tokens) is returned.



# |
# |
# |
# |


-> model = BigramLanguageModel(vocab_size=500)
m = model.to(device)

BigramLanguageModel(vocab_size=500) → Initializes the model with a vocabulary size of 500.

Each token's embedding will have 500 dimensions.
The embedding table is also trained to predict the next token in the sequence.

-->Step 2: Creating the context Tensor

context = torch.zeros((1, 1), dtype=torch.long, device=device)
✅ This creates a tensor filled with zeros:

context = tensor([[0]])
(1, 1) → A batch size of 1 and a sequence length of 1 (starting prompt is a single token).
dtype=torch.long → Since token indices are integers, long is used.
device=device → Ensures the tensor is on the same device as the model.
🧠 Why Use 0 as the Starting Token?

The starting token is often zero when no prompt is provided.
It's a placeholder token, and the model will rely on learned embeddings to generate meaningful text from there.


-->Step 3: Generating Tokens

generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())

Inside generate()
The context tensor is passed into m.generate() with max_new_tokens=500.
The model will predict and generate 500 additional tokens.
[0] → Extracts the sequence from the first batch (since batch size is 1).
.tolist() → Converts the tensor to a Python list for easier decoding.


-->Example Output from .generate()
tensor([[0, 123, 98, 45, 200, 302, ... , 76]])

The sequence now contains the starting token 0 followed by 500 generated tokens.


-->Step 4: Decoding the Generated Tokens

generated_chars = decode(...)
✅ The decode() function likely converts the list of token indices back into readable text.

Example of decode() function logic:


def decode(token_list):
    return ''.join([vocab[idx] for idx in token_list])
The vocab variable would be a mapping like:

vocab = {0: "<START>", 1: "H", 2: "e", 3: "l", 4: "o", ...}
For example:

[0, 1, 2, 3, 3, 4] → "<START>Hello"

--> Step 5: Printing the Result

print(generated_chars)
✅ Displays the generated text output, which could look something like:

"Hello world! This is a sample text generated by the Bigram model..."


--> Complete Flow Summary
Model Initialization: The model is created with vocab_size = 500.
Context Creation: A starting token (0) is provided.
Token Generation: The generate() method repeatedly predicts and appends new tokens for 500 steps.
Decoding: The sequence of token indices is converted into readable text.
Output Display: The generated text is printed.



Key Concepts to Remember
✅ logits[:, -1, :] extracts the last token's logits for predicting the next token.
✅ torch.softmax() converts logits into probabilities.
✅ torch.multinomial() samples tokens randomly, adding diversity to the generated text.
✅ torch.cat() appends the predicted token to the existing sequence.

#NEW

# Training loop

In [ ]:
optimizer = torch.optim.AdamW(model.parameters(),lr=learning_rate)

for iter in range(max_iters):
    
    #sample a batch of data
    xb,yb = get_batch('train')
    xb, yb = xb.to(device), yb.to(device)   # Move data to GPU
    
    #evaluate the loss
    
    logits , loss = model.forward(xb,yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

1.7047104835510254
